In [1]:
from functools import partial
from itertools import combinations, repeat
from craterdetection.common.conics import conic_center

from craterdetection.matching.database import CraterDatabase, load_craters, extract_robbins_dataset, crater_representation, CoplanarInvariants
from craterdetection.matching.utils import get_cliques_by_length
from astropy.coordinates import spherical_to_cartesian, cartesian_to_spherical
from craterdetection.common.coordinates import ENU_system, nadir_attitude
from craterdetection.common.camera import *
import craterdetection.common.constants as const

import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as LA
import networkx as nx
from sklearn.neighbors import radius_neighbors_graph
from scipy.spatial.transform import Rotation
from scipy.spatial.distance import cdist

from craterdetection.matching.utils import is_clockwise, np_swap_columns, all_clockwise, is_colinear, cw_or_ccw, \
    enhanced_pattern_shifting


In [4]:
lat_cat, long_cat, major_cat, minor_cat, psi_cat, crater_id_cat = extract_robbins_dataset(
    load_craters("../data/lunar_crater_database_robbins_2018.csv", diamlims=[4, 30], latlims=[20, 70], longlims=[20, 70])
)

In [8]:
x_cat, y_cat, z_cat = map(np.array, spherical_to_cartesian(const.RMOON, lat_cat, long_cat))

adjacency_matrix = radius_neighbors_graph(np.array([x_cat, y_cat, z_cat]).T, 150,
                                                                  mode='distance',
                                                                  metric='euclidean', n_jobs=-1)

graph = nx.from_scipy_sparse_matrix(adjacency_matrix)
crater_triads = np.array(get_cliques_by_length(graph, 3))

In [ ]:
p_M_ijk = np.moveaxis(
    np.concatenate(
        (x_cat[crater_triads].T[None, ...],
         y_cat[crater_triads].T[None, ...],
         z_cat[crater_triads].T[None, ...]),
        axis=0
    ),
    0, 2)[..., None]
p_centroid = np.mean(p_M_ijk, axis=0)

In [ ]:
# VIRTUAL CAMERA
alt = 300
p_vcam = p_centroid + (p_centroid/LA.norm(p_centroid, axis=1)[..., None])*alt

X_ax_cam, Y_ax_cam, Z_ax_cam = nadir_attitude(p_vcam)
T_CM = np.concatenate((X_ax_cam, Y_ax_cam, Z_ax_cam), axis=-1)
if (LA.matrix_rank(T_CM) != 3).any():
    raise Warning("Invalid camera attitude matrices present!:\n", T_CM)

In [ ]:
K = camera_matrix()
T_MC = LA.inv(T_CM)
P_MC = K @ T_MC @ np.concatenate((np.tile(np.identity(3), (len(p_vcam), 1, 1)), -p_vcam), axis=2)

In [ ]:
C_cat = crater_representation(major_cat, minor_cat, psi_cat)
H_C_triads = np.array(list(map(crater_camera_homography, p_M_ijk, repeat(P_MC))))

C_triads = np.array(list(map(lambda vertex: C_cat[vertex], crater_triads.T)))
A_i, A_j, A_k = map(lambda T, C: LA.inv(T).transpose((0, 2, 1)) @ C @ LA.inv(T), H_C_triads, C_triads)

In [ ]:
r_i, r_j, r_k = map(conic_center, (A_i, A_j, A_k))
cw_value = LA.det(np.moveaxis(np.array([[r_i[:, 0], r_i[:, 1], np.ones_like(r_i[:, 0])],
                            [r_j[:, 0], r_j[:, 1], np.ones_like(r_i[:, 0])],
                            [r_k[:, 0], r_k[:, 1], np.ones_like(r_i[:, 0])]]), -1, 0))
clockwise = cw_value < 0
line = cw_value == 0

In [ ]:
clockwise = clockwise[~line]

crater_triads = crater_triads[~line]
crater_triads[np.argwhere(~clockwise), [0, 1]] = crater_triads[np.argwhere(~clockwise), [1, 0]]

H_C_triads = H_C_triads[:, ~line]
H_C_triads[[0, 1], np.argwhere(~clockwise)] = H_C_triads[[1, 0], np.argwhere(~clockwise)]

In [ ]:
C_triads = np.array(list(map(lambda vertex: C_cat[vertex], crater_triads.T)))
A_i, A_j, A_k = map(lambda T, C: LA.inv(T).transpose((0, 2, 1)) @ C @ LA.inv(T), H_C_triads, C_triads)

In [ ]:
def plot_conics(A, gridsize=250, resolution=const.CAMERA_RESOLUTION, figsize=(15, 15), ax=None):
    x_plot = np.linspace(0, resolution[0], gridsize)
    y_plot = np.linspace(0, resolution[1], gridsize)
    x_plot, y_plot = np.meshgrid(x_plot, y_plot)

    xy_homogeneous = np.concatenate(
        (
            x_plot.ravel()[None, :],
            y_plot.ravel()[None, :],
            np.ones_like(x_plot.ravel()[None, :])
        ),
        axis=0
    ).T[..., None]
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize, subplot_kw={'aspect': 'equal'})

    # Set axes according to camera pixel space convention
    ax.set_xlim(0, resolution[0])
    ax.set_ylim(resolution[1], 0)

    for a_i in A:
        c = xy_homogeneous.transpose(0, 2, 1) @ a_i @ xy_homogeneous
        ax.contour(x_plot, y_plot, c.reshape(x_plot.shape), [0], colors='r')
i = 782193
plot_conics([A_i[i], A_j[i], A_k[i]])

In [ ]:
test_triad = crater_triads[i]
avg_lat, avg_long = map(np.mean, (lat_cat[test_triad], long_cat[test_triad]))

In [ ]:
cam1 = Camera.from_coordinates(avg_lat, avg_long+0.1, 100)
cam1.T = (Rotation.from_matrix(cam1.T) * Rotation.from_euler('y', -30, degrees=True)).as_matrix()

A_detections1 = cam1.project_crater_conics(C_cat[test_triad], p_M_ijk[:, i])
plot_conics(A_detections1)
CoplanarInvariants.from_detection_conics(A_craters=A_detections1).get_pattern()

In [ ]:
cam2 = Camera.from_coordinates(avg_lat-0.2, avg_long, 300)
cam2.T = (Rotation.from_matrix(cam2.T) * Rotation.from_euler('x', 30, degrees=True)).as_matrix()
cam2.T = (Rotation.from_matrix(cam2.T) * Rotation.from_euler('z', 15, degrees=True)).as_matrix()

A_detections2 = cam2.project_crater_conics(C_cat[test_triad], p_M_ijk[:, i])
plot_conics(A_detections2)
CoplanarInvariants.from_detection_conics(A_craters=A_detections2).get_pattern()

In [ ]:
CoplanarInvariants.from_detection_conics(A_craters=A_detections2).crater_triads

In [ ]:
crater_triads_eff = np.zeros((5000000, 3))*-1
for i, c in enumerate((c for c in nx.cycle_basis(graph) if len(c) == 3)):
    crater_triads_eff[i] = c
crater_triads_eff = crater_triads_eff[crater_triads_eff != -1]
crater_triads_eff

In [ ]:
crater_triads_eff

In [ ]:
def get_cliques_by_length(G, length_clique):
    """ Return the list of all cliques in an undirected graph G with length
    equal to length_clique. """
    cliques = []
    for c in nx.enumerate_all_cliques(G) :
        if len(c) <= length_clique:
            if len(c) == length_clique:
                cliques.append(c)
        else:
            return cliques
    # return empty list if nothing is found
    return cliques

get_cliques_by_length(graph, 3)